# Dashboarding

To turn our workflow into a simple dashboard, all we need to do is add some ipywidgets. 

## Import ipywidgets
Let's add an import statement for ipywidgets. We will use a lot of widgets, so it's good to import ipywidgets as widgets, or something similiarly short. Add an import statement below and then go to [ipywidgets documentation site](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) to get a sense of what kind of widgets there are.

In [ ]:
import pandas as pd
import os
import ipywidgets as widgets # add import statement
from matplotlib import pyplot as plt

## Load Data

Now we will load the data, just like last time.

In [ ]:
# Load data into memory from file
DATA_DIR = 'data'
DATA_FILE = 'land-ocean-temp-index.csv'

df = pd.read_csv(os.path.join(DATA_DIR, DATA_FILE), escapechar='#')
df

## Select Data

Okay. Now we want to select the data and also update the display of the dataframe. To do this, we will use our first ipywidget. Because we are selecting a range of years, we will use the `IntRangeSlider` widget, which has two click and drag nobs. Let's see what that looks like and play around with the click-and-drag functionality.

In [ ]:
slider = widgets.IntRangeSlider()
slider

Okay great! Plat But 0 - 100 isn't the range we want. Instead, we would like the minimum and maximum to be the earliest and latest years in our dataset. So we need to obtain those first. 

In [ ]:
(min(df['Year']), max(df['Year']))

That looks right. Let's use the same syntax to set the min and max of the slider widget.

In [ ]:
slider.max = max(df['Year']) # set the 'max' attribute of the slider to the minimum year of the our data  
slider.min = min(df['Year']) #   and let's do the same for 'min'

Okay that looks a lot better, but how to we actually get the value of the range from the widget? Generally, most widgets have a `value` attribute, but since there are actually two values in a range, I'm not sure what to expect. Let's see what happens when we try and poll the value attribute. What data type is it?

In [ ]:
slider.value

That looks like a tuple. Let's try setting the value programmatically as well.

In [ ]:
slider.value = (1900, 2000)

Okay, now that we can get the range from the slider, how do we relect those selections back to the dataframe? Well, we probably want to keep two copies, because if we change the original to have a shorter range, and then we want to expand that range, we're going to have lost data. So let's maintain a dataframe called `selected` that reflects the range specified by the Slider. Let's look at how we did that in workflow.ipynb. 

```python
selected = df[(df['Year'] >= from_year) & (df['Year'] <= to_year)]
```

The only difference is this time, we need to replace "from_year" and "to_year" with the value of our slider. Try that on your own in the cells below.

In [ ]:
selected = df[(df['Year'] >= slider.value[0]) & (df['Year'] <= slider.value[1])] # selected = df[(df['Year'] >= from_year) & (df['Year'] <= to_year)]
selected

Okay, that's a start. But now we want to do this dynamically. That is, we want the line above to be called every time the range is modified. How do we track changes to the value of the slider?

## Observe function

Widgets are built on the `traitlets` library, which we will learn more about later on. Attributes of objects that inherit the `HasTraits` class, such as widgets do, are called traits. Any trait can be "observed" for changes. This means we can observe the "value" trait of the slider for changes.

In [ ]:
def on_slider_change(change):
    print(change)

In [ ]:
slider.observe(on_slider_change, 'value')

So if we change the value of the slider, our observe function will print the "change". But what type of data is "change"? Let's find out by changing the value of the slider programmatically.

In [ ]:
slider.value = (1880, 2000)

So there are a lot of things going on here. Now we know that `change` is a dictionary. There are a lot of key/value pairs here, but the most important one's for us right now are the "old" and "new" keys. We can use them to see what the value trait used to be, and what it was updated to. 

## Update `selected` dataframe with range
Now that we know how to observe changes in the slider value, we want to use that observe function to update the `selected` dataframe. Let's rewrite the `on_range_change` function so that it updates the `selected` dataframe.

In [ ]:
def on_slider_change(change):
    global selected # selected
    selected = df[(df['Year'] >= change['new'][0]) & (df['Year'] <= change['new'][1])]

In [ ]:
slider.observe(on_slider_change, 'value')

In [ ]:
slider.value = (1990, 1993)
selected

That worked really well! But we are still printing the change. I thought we redefined the function! As it turns out, both versions of the function have copies that are called when there is a change. To get rid of all those observe functions, we use the `unobserve_all` method. Go ahead and try that.

In [ ]:
slider.unobserve_all()

In [ ]:
slider.observe(on_slider_change, 'value') # slider.observe(on_slider_change)
slider.value = (1993, 1996)
slider.value = (1996, 1999)

Hmmmm.... it seems that we've gotten an error. Sort of a weird one. I think we are missing something on our observe call. Can you think of what it is? Maybe scroll up and look and the last observe call to see what's different. Then fix the call of the observe method and rerun the two cells above. 

Does it run like it's supposed to now? Great! Now that we have that down, the next step is to update the dataframe display every time the range is changed. But how do we achieve that? The dataframe isn't actually a widget yet. It's just being displayed with ipython. But we can turn anything that can be displayed with ipython into a widget with Output.

## Output widgets

First, let's see what output looks like on it's own.

In [ ]:
output_df = widgets.Output()
output_df

Pretty boring. **Let's right click on the Output widget and "Create new view for output"** for now. How do we get it to display our dataframe? We use the Output widget as a context manager. Anything that we can display with ipython.display can be displayed by the Output widget using the following syntax.

In [ ]:
with output_df:
    display(df)

Voila! The output_df widget now displays the dataframe. Now we need to be able to update that display in case someone changes the range. Let's write a function for this.

In [ ]:
def display_df(change): 
    output_df.clear_output(wait=True)
    with output_df: 
        display(selected)

In [ ]:
display_df(None)

Cool! Now the only thing that is missing is that we want to call this function every time the range is updated. How might we go about that?

Well, we could use another observe function! Let's try it.

In [ ]:
slider.observe(display_df, 'value')

Okay let's create another copy of the slider so we can change the value that way. Then go ahead and move the range around. 

In [ ]:
slider

Way cool. Now our dataframe reflects the range of values.

## Add column for Savitzky-Golay filter

Now we have data selection down, so the next step is to also add a column to our data for the smoothed curve. Let's review what we had in the workflow. Keep in mind that we will be working with `df` instead of `selected` for a while.

In [ ]:
from scipy.signal import savgol_filter

# Set the window size and polynomial order for the Savitzky-Golay filter
window_size = 20
poly_order = 3

# Apply Savitzky-Golay smoothing to the Temperature column
df['Savitzky-Golay'] = savgol_filter(df['Temperature'], window_size, poly_order)
df

Okay. So we have these two variables that we want to turn into sliders. Let's make the Sliders first.

In [ ]:
window_size = widgets.IntSlider(description = 'Window Size', min=1, max=100)
window_size

In [ ]:
poly_order = widgets.IntSlider(description = 'Poly Order', min=0, max=10)
poly_order

Now when we update window size, we want a few things to happen. 

1. Update df
2. Use df to update selected
3. Update display of selected

Let's write a function that takes care of the first two.

In [ ]:
def on_window_size_change(change):
    global df, selected
    df['Savitzky-Golay'] = savgol_filter(df['Temperature'], change['new'], poly_order.value) #df['Savitzky-Golay'] = savgol_filter(df['Temperature'], window_size, poly_order)
    selected = df[(df['Year'] >= slider.value[0]) & (df['Year'] <= slider.value[1])]

Then do you recall what function we used to display the dataframe? It was `display_df`. Let's make sure we add both to the functions observed by window_size.

In [ ]:
window_size.observe(on_window_size_change, 'value')
window_size.observe(display_df, 'value')

In [ ]:
window_size

Sweet! If you still have an Output View of the `selected` dataframe open, and you increase the value of `window_size` you will see that there is now a Savitzky-Golay column. Now I want you to create a Slider widget that is linked to the `poly_order`. 

In [ ]:
def on_poly_order_change(change):
    global df, selected
    df['Savitzky-Golay'] = savgol_filter(df['Temperature'], window_size.value, change['new']) #df['Savitzky-Golay'] = savgol_filter(df['Temperature'], window_size, poly_order)
    selected = df[(df['Year'] >= slider.value[0]) & (df['Year'] <= slider.value[1])]

In [ ]:
poly_order.observe(on_poly_order_change, 'value')
poly_order.observe(display_df, 'value')
poly_order

That looks pretty good! Let's try setting both values programmatically.

In [ ]:
window_size.value = 3
poly_order.value = 4

Uh oh. Why did this happen? Fortunately the error message is pretty clear. We need to ensure that our poly_order Slider doesn't allow a user to select a value greater than `window_length - 1`. But we also don't want to let poly_order be greater than 10 to avoid poor conditioning. Where should we put this validation? Well, we are going to need to update the "max" trait of `poly_order` every time `window_length` is updated. So we need to add a line to add a line in the `on_window_length_change` funciton.

In [ ]:
def on_window_size_change(change):
    global df, selected, poly_order
    poly_order.max = min(change['new'] - 1, 10) # change the maximum of the poly_order widget
    df['Savitzky-Golay'] = savgol_filter(df['Temperature'], change['new'], poly_order.value)
    selected = df[(df['Year'] >= slider.value[0]) & (df['Year'] <= slider.value[1])]


In [ ]:
window_size.unobserve_all()
window_size.observe(on_window_size_change, 'value')
window_size.observe(display_df, 'value')
window_size

In [ ]:
poly_order

Amazing! We are doone visualizing our selected data. Now we can move on to visualizing our data with a plot.

## Visualizing Data

We will use matplotlib, because the syntax is so common, but there are more interactive widgets in the ipywidget ecosystem that would work even better, such as bqplot.

Before we get started, let's take a look at what we had in the original workflow.

```python
plt.xlabel('Year')
plt.ylabel('Temperature')
plt.plot(selected_range['Year'], selected_range['Temperature'])
plt.plot(selected_range['Year'], selected_range['Savitzky-Golay']) 
plt.show()
```

We are going to want to turn this into an observe function that accepts a "change" parameter. Is there anything we need to fix here to make this work? It's very minor. Edit the code chunk below so that it works in our current notebook. 

In [ ]:
slider.value = (1880, 2009)

plt.xlabel('Year')
plt.ylabel('Temperature')
plt.plot(selected['Year'], selected['Temperature']) # plt.plot(selected_range['Year'], selected_range['Temperature'])
plt.plot(selected['Year'], selected['Savitzky-Golay']) # plt.plot(selected_range['Year'], selected_range['Savitzky-Golay']) 
plt.show()

Okay that works, but the plot is showing up in the log, and we want to show it in our notebook. What can we do to show and update the plot displayed in the notebook?

Yeah, we can use an Output widget again, just like we did with the dataframe.

In [ ]:
output_plot = widgets.Output() # create an output widget called output_plot
output_plot

Go ahead and create an Output View for this widget also.

Okay. Now let's remember what we did to update our display last time. 
```python
def display_df(change): 
    output_df.clear_output(wait=True)
    with output_df: 
        display(selected)
```

Go ahead and adapt this function so that instead of displaying the selected dataframe, it displays an updated plot.

In [ ]:
def display_plot(change): #def display_df(change): 
    output_plot.clear_output(wait=True) #output_df.clear_output(wait=True)
    with output_plot: 
        plt.xlabel('Year') # display(selected)
        plt.ylabel('Temperature') #
        plt.plot(selected['Year'], selected['Temperature']) #
        plt.plot(selected['Year'], selected['Savitzky-Golay']) # 
        plt.show() #

Let's make sure this function gets called any time there is a change to our selected data or to the Savitzky-Golay parameters. How do we make sure this function runs every time there is a change?

In [ ]:
slider.observe(display_plot, 'value')
window_size.observe(display_plot, 'value')
poly_order.observe(display_plot, 'value')

In [ ]:
slider

In [ ]:
window_size

In [ ]:
poly_order

Wow that looks amazing! We have a total of five widgets. Three of them control various parameters and they have dependencies on each other. In the next notebook, we will see how this can all come together into one very beautiful interface.